In [18]:
import pandas as pd
import numpy as np
import glob

In [19]:
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

### 강원 기온 전처리

#강원 기온
east = pd.read_csv('temp_rain/강원영동_기온.csv',skiprows=7, encoding='cp949')
west = pd.read_csv('temp_rain/강원영서_기온.csv',skiprows=7, encoding='cp949')

new = pd.DataFrame()

new['날짜'] = east['날짜']
new['지점'] = '강원'
new['평균기온(℃)'] = (east['평균기온(℃)']+west['평균기온(℃)'])/2
new['최저기온(℃)'] = (east['최저기온(℃)']+west['최저기온(℃)'])/2
new['최고기온(℃)'] = (east['최고기온(℃)']+west['최고기온(℃)'])/2
new

new.to_csv('강원_기온.csv',index=False,encoding='cp949')

#강원 강수

east = pd.read_csv('temp_rain/강원영동_강수.csv',skiprows=7, encoding='cp949')
west = pd.read_csv('temp_rain/강원영서_강수.csv',skiprows=7, encoding='cp949')

new = pd.DataFrame()

new['날짜'] = east['날짜']
new['지점'] = '강원'
new['강수량(mm)'] = (east['강수량(mm)']+west['강수량(mm)'])/2
new

new.to_csv('강원_강수.csv',index=False,encoding='cp949')

### 데이터 병합

In [20]:
train_temp= glob.glob('C:/Users/juyeo/Desktop/03. 농산물 가격예측 AI 경진대회/temp_rain/*_기온.csv')
train_rain= glob.glob('C:/Users/juyeo/Desktop/03. 농산물 가격예측 AI 경진대회/temp_rain/*_강수.csv')

onion = pd.read_csv('data_onion.csv')
cabbage = pd.read_csv('data_cabbage.csv')

In [21]:
def merge_train(argi,weather):
    new = pd.DataFrame()

    for i in argi['지점'].unique():
        for train in weather:
            if i == train[-9:-7] or i == train[-11:-7]:
                unit = pd.read_csv(train,skiprows=7, encoding='cp949')
                con = pd.merge(argi[argi['지점']==i],unit,on=['날짜','지점'])
                new = pd.concat([new,con],axis=0)

    return new

In [22]:
def make_argi(argi):
    temp = merge_train(argi,train_temp)
    rain = merge_train(argi,train_rain)
    temp['강수량(mm)'] = rain['강수량(mm)']
    data = temp.groupby(['날짜']).mean()
    return data

In [25]:
onion = make_argi(onion)
cabbage = make_argi(cabbage)

In [28]:
onion.to_csv('onion.csv',index=True)
cabbage.to_csv('cabbage.csv',index=True)